# Setup

In [ ]:
%uv pip install -e ..[dev,gpu]

# 🤖 Focoos Foundational Models


In [ ]:
import os
from pprint import pprint

from focoos import DEV_API_URL, Focoos

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=DEV_API_URL)

pprint(focoos.list_focoos_models())

## Cloud Inference

In [ ]:
import os
from pprint import pprint

from supervision import plot_image

from focoos import DEV_API_URL, Focoos

model_ref = "focoos_object365"
image_path = "./assets/ade_val_034.jpg"

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=DEV_API_URL)

model = focoos.get_remote_model(model_ref)
## Only admin can deploy foundational models

output, preview = model.infer(image_path, threshold=0.4, annotate=True)
plot_image(preview)

## Local Inference

This section demonstrates how to perform local inference using a model from the Focoos platform. 
We will load a model, deploy it locally, and then run inference on a sample image.


### Available Runtime Types

In [ ]:
from focoos.ports import RuntimeTypes

for runtime_type in RuntimeTypes:
    print(runtime_type)

### OnnxRuntime With CUDA (focoos_object365)

In [ ]:
import os
from pprint import pprint

from supervision import plot_image

from focoos import DEV_API_URL, Focoos

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=DEV_API_URL)
image_path = "./assets/ade_val_034.jpg"
model_ref = "focoos_object365"


model = focoos.get_local_model(model_ref)

latency = model.benchmark(iterations=10, size=640)
pprint(latency)
# pprint(latency)
output, preview = model.infer(image_path, threshold=0.3, annotate=True)
pprint(output.detections)
pprint(output.latency)

plot_image(preview)

### OnnxRuntime With TensorRT (FP16) (focoos_object365)

In [ ]:
import os
from pprint import pprint

from supervision import plot_image

from focoos import Focoos
from focoos.ports import RuntimeTypes

focoos = Focoos(
    api_key=os.getenv("FOCOOS_API_KEY"),
)
image_path = "./assets/ade_val_034.jpg"
model_ref = "focoos_object365"


model = focoos.get_local_model(model_ref, runtime_type=RuntimeTypes.ONNX_TRT16)

latency = model.benchmark(iterations=10, size=640)
pprint(latency)
# pprint(latency)
output, preview = model.infer(image_path, threshold=0.3, annotate=True)
pprint(output.detections)
pprint(output.latency)

plot_image(preview)

# User Models

### List User Models

In [ ]:
import os
from pprint import pprint

from dotenv import load_dotenv

from focoos import DEV_API_URL, Focoos

load_dotenv()

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=DEV_API_URL)

models = focoos.list_models()
pprint(models)

## Create Model

In [ ]:
import os
from pprint import pprint

from dotenv import load_dotenv

from focoos import DEV_API_URL, Focoos

load_dotenv()

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=DEV_API_URL)

model = focoos.new_model(name="test-model", focoos_model="focoos_object365", description="Test model")
### Get Model Info

## Deploy user model on shared cloud endpoint

In [ ]:
import os
from pprint import pprint

from dotenv import load_dotenv

from focoos import DEV_API_URL, Focoos

load_dotenv()

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=DEV_API_URL)

model = focoos.get_remote_model(os.getenv("FOCOOS_MODEL_REF"))
model_info = model.get_info()

## Cloud Inference

In [ ]:
import os
from pprint import pprint

import supervision as sv
from dotenv import load_dotenv

from focoos import Focoos

load_dotenv()

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=DEV_API_URL)

model = focoos.get_remote_model(os.getenv("FOCOOS_MODEL_REF"))
model_info = model.get_info()
image_path = "./assets/aquarium.jpg"

output, preview = model.infer(image_path, threshold=0.5, annotate=True)
sv.plot_image(preview)

## Local Inference




In [ ]:
import os
from pprint import pprint

from supervision import plot_image

from focoos import DEV_API_URL, Focoos

focoos = Focoos(
    api_key=os.getenv("FOCOOS_API_KEY"),
    host_url=DEV_API_URL,
)
image_path = "./assets/aquarium.jpg"
model_ref = "focoos_object365"

model_info = focoos.get_model_info(model_ref)

model = focoos.get_local_model(model_ref)

latency = model.benchmark(iterations=10, size=640)
# pprint(latency)
output, preview = model.infer(image_path, threshold=0.3, annotate=True)

plot_image(preview)

## Train a Model

### List Public Datasets


In [ ]:
import os
from pprint import pprint

from focoos import DEV_API_URL, Focoos

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=DEV_API_URL)

datasets = focoos.list_shared_datasets()
pprint(datasets)
dataset_ref = "f92ac3b33b284a80"

In [ ]:
import os

from focoos.ports import Hyperparameters, TrainInstance

model = focoos.new_model(name="test-model-4", focoos_model="focoos_coco_tiny", description="Test model 4")
model_ref = model.model_ref
res = model.train(
    dataset_ref=dataset_ref,
    instance_type=TrainInstance.ML_G4DN_XLARGE,
    volume_size=50,
    max_runtime_in_seconds=36000,
    hyperparameters=Hyperparameters(
        learning_rate=0.0001,
        batch_size=32,
        max_iters=5000,
        eval_period=100,
        resolution=640,
    ),  # type: ignore
)

## Monitor Training

In [ ]:
import os

from focoos import DEV_API_URL, Focoos

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=DEV_API_URL)
model = focoos.get_remote_model("focoos_object365")
model.notebook_monitor_train(interval=30, plot_metrics=True)

In [ ]:
logs = model.train_logs()
pprint(logs)

## Get User Info

In [ ]:
import os
from pprint import pprint

from focoos import DEV_API_URL, Focoos

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=DEV_API_URL)

user_info = focoos.get_user_info()
pprint(user_info.model_dump())

## Get System Info

In [ ]:
from focoos.utils.system import get_system_info

system_info = get_system_info()
system_info.pretty_print()

### Model Metrics




In [ ]:
import os
from pprint import pprint

from focoos import LOCAL_API_URL, Focoos

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=LOCAL_API_URL)

model = focoos.get_remote_model(os.getenv("FOCOOS_MODEL_REF"))
metrics = model.metrics()

## Visualize Metrics


In [ ]:
from focoos.utils.metrics import MetricsVisualizer

metrics_visualizer = MetricsVisualizer(metrics)
metrics_visualizer.log_metrics()
metrics_visualizer.notebook_plot_training_metrics()